<a href="https://colab.research.google.com/github/Vnnie-Mun/AI-ML_full-projects_repository/blob/main/sentiment_Analysis_on_IMDB_Reviews_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install kaggle


Import the dependancies

In [36]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

Data collection

In [10]:
kaggle_dictionary = json.load(open("/content/kaggle (4).json"))


In [11]:
#setup kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]


In [12]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


In [13]:
    import zipfile

In [14]:
        from google.colab import drive
        drive.mount('/content/drive/')

Mounted at /content/drive/


In [15]:
zip_file_path = '/content/archive.zip'
extraction_path = '/content/drive/MyDrive/imdb_dataset'

In [16]:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

In [17]:
!ls

 archive.zip   drive  'kaggle (4).json'   sample_data


Loading the Data set

In [18]:
data = pd.read_csv("/content/drive/MyDrive/imdb_dataset/IMDB Dataset.csv")

In [19]:
!ls

 archive.zip   drive  'kaggle (4).json'   sample_data


In [20]:
data.shape

(50000, 2)

In [21]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [22]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [23]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [24]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

/tmp/ipython-input-2568826810.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [25]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [26]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [27]:
#splitting the data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [28]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data preprocessing

In [29]:
#tokenize the text data
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [30]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [31]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [32]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [33]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


In [34]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


LSTM -Long short Term Memory

In [37]:
#build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [39]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
#compile the model
model.compile(optimizer="adam",loss="binary_crossentropy", metrics=["accuracy"])

Training the model

In [42]:
model.fit(X_train,Y_train,epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 228s 441ms/step - accuracy: 0.7307 - loss: 0.5302 - val_accuracy: 0.8254 - val_loss: 0.4218
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 242s 410ms/step - accuracy: 0.8436 - loss: 0.3693 - val_accuracy: 0.8605 - val_loss: 0.3454
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 252s 390ms/step - accuracy: 0.8699 - loss: 0.3189 - val_accuracy: 0.8378 - val_loss: 0.3663
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 390ms/step - accuracy: 0.8879 - loss: 0.2826 - val_accuracy: 0.8733 - val_loss: 0.3240
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 195s 390ms/step - accuracy: 0.9051 - loss: 0.2414 - val_accuracy: 0.8618 - val_loss: 0.3557


MOdel evaluation

In [43]:
loss, accuracy=model.evaluate(X_test,Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 113ms/step - accuracy: 0.8682 - loss: 0.3437
Test Loss: 0.3357697129249573
Test Accuracy: 0.8700000047683716


BUilding a predictive model

In [44]:
def predict_sentiment(review):
  #tokenize and pad the review
  sequence = tokenizer.texts_to_sequences(review)
  padded_sequence = pad_sequences(sequence, maxlen=200)

  #make a prediction
  prediction = model.predict(padded_sequence)
  sentiment ="positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [45]:
#example usage
new_review="This movie was fantastic. Iloved it"
sentiment = predict_sentiment([new_review])
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
The sentiment of the review is: positive


In [46]:
#example usage
new_review="This movie was fantastic. Iloved it"
sentiment = predict_sentiment([new_review])
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
The sentiment of the review is: positive
